### Dependencias

In [2]:
import cupy as cp
import numpy as np
import pandas as pd

### Función de activación

In [3]:
def sigmoid(x):
    return 1 / (1 + cp.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def softmax(z):#z: vector resultante antes de funcion de activación en la última capa
    exp_z = cp.exp(z - cp.max(z, axis=1, keepdims=True))  
    return exp_z / cp.sum(exp_z, axis=1, keepdims=True)#retorna las probabilidades de las posibles clases

#mide la diferencia entre distribucion de prob. creada por softmax y los targets reales
def cross_entropy_loss(predictions, targets):
    return -cp.sum(targets * cp.log(predictions + 1e-9)) / targets.shape[0]

#### Encoder que cambia el formato de labels tal que coincida con las predicciones

In [4]:
def one_hot(y, num_classes):
    if y.ndim > 1:  # Flatten the array if necessary
        y = y.flatten()
        
    one_hot_labels = cp.zeros((y.shape[0], num_classes))
    one_hot_labels[cp.arange(y.shape[0]), y] = 1
    return one_hot_labels

### Clase MLP

In [11]:

class MultiLayerNetwork:
    def __init__(self, layer_sizes):
        self.layer_sizes = layer_sizes
        self.weights = []
        self.biases = []

        # Initialize weights and biases for each layer
        for i in range(len(layer_sizes) - 1):
            weight = cp.random.randn(layer_sizes[i], layer_sizes[i + 1]) * cp.sqrt(2. / layer_sizes[i])
            bias = cp.zeros((1, layer_sizes[i + 1]))
            self.weights.append(weight)
            self.biases.append(bias)

    def forward(self, inputs):
        self.activations = [inputs]
        a = inputs

        for i in range(len(self.weights) - 1):
            z = cp.dot(a, self.weights[i]) + self.biases[i]
            a = sigmoid(z)
            self.activations.append(a)

        # Output uses softmax for multiclass classification
        z = cp.dot(a, self.weights[-1]) + self.biases[-1]
        a = softmax(z)
        self.activations.append(a)
        return a

    def predict(self, inputs):
        inputs = cp.array(inputs, ndmin=2)
        a = inputs

        for i in range(len(self.weights) - 1):
            z = cp.dot(a, self.weights[i]) + self.biases[i]
            a = sigmoid(z)
        
        # Output with softmax
        z = cp.dot(a, self.weights[-1]) + self.biases[-1]
        a = softmax(z)

        return a

    def backward(self, targets, learning_rate):
        m = targets.shape[0]  # number of training examples
        delta_weights = [0] * len(self.weights)
        delta_biases = [0] * len(self.biases)

        # Calculate the initial error (difference between prediction and target for the output layer)
        error = self.activations[-1] - targets

        for i in reversed(range(len(self.weights))):
            # Calculate the delta for the current layer
            delta = error
            delta_weights[i] = cp.dot(self.activations[i].T, delta) / m
            delta_biases[i] = cp.sum(delta, axis=0, keepdims=True) / m

            if i != 0:
                # Propagate the error to the previous layer
                error = cp.dot(delta, self.weights[i].T) * sigmoid_derivative(self.activations[i])

            # Update weights and biases
            self.weights[i] -= learning_rate * delta_weights[i]
            self.biases[i] -= learning_rate * delta_biases[i]

            
    def train(self, inputs, targets, epochs, learning_rate):
        targets = one_hot(targets, 10)

        errors = []

        for epoch in range(epochs):
            predictions = self.forward(inputs)
            error = cross_entropy_loss(predictions, targets)
            self.backward(targets, learning_rate)
            errors.append(error)
            
            if epoch % 10 == 0:
                print(f'Epoch {epoch}, Error: {error}')

        return errors

### Data Prep

In [6]:
#Obtener datos (estoy usando pd porque anda considerablemente más rápido que np)
train_data = pd.read_csv(r'C:/Users/kueru/Documents/VSCode/semestre_9/Deep_Learning/T2/train_data_2.csv')
train_data = train_data.to_numpy()
    
#Cortar en features y labales
train_samples = train_data.shape[0]
features = train_data[:train_samples, 1:-1]  # Features for training    
labels = train_data[:train_samples, -1]  #Labels for training

labels = labels.reshape(-1, 1)  # Reshape to (299, 1)

X_train = cp.array(features)
y_train = cp.array(labels).flatten()

print(train_data.shape)
print(features.shape)
print(labels.shape)

(10155, 3074)
(10155, 3072)
(10155, 1)


### Train

In [12]:
num_classes = 10
input_size = 3072
hidden_layers = [256,256,256]  # Tamaños de las capas ocultas
output_size = 10
layer_sizes = [input_size] + hidden_layers + [output_size]

model = MultiLayerNetwork(layer_sizes)
epochs = 300
learning_rate = 0.05

errors = model.train(X_train, y_train, epochs, learning_rate)
#Evaluar
predictions = model.predict(X_train)
accuracy = cp.mean(predictions.argmax(axis=1) == y_train)
print(f'Test Accuracy: {accuracy}')

Epoch 0, Error: 2.5068105629092963
Epoch 10, Error: 2.28947607864776
Epoch 20, Error: 2.2827386220391017
Epoch 30, Error: 2.277174301808653
Epoch 40, Error: 2.2720083767745565
Epoch 50, Error: 2.266968359081273
Epoch 60, Error: 2.2618555014493555
Epoch 70, Error: 2.256966304416771
Epoch 80, Error: 2.2516349427355724
Epoch 90, Error: 2.245993515141124
Epoch 100, Error: 2.240669595078174
Epoch 110, Error: 2.2348192600822943
Epoch 120, Error: 2.229231993656745
Epoch 130, Error: 2.2232739632544956
Epoch 140, Error: 2.216925876907649
Epoch 150, Error: 2.2106246246245465
Epoch 160, Error: 2.2035434754232464
Epoch 170, Error: 2.1967659979894103
Epoch 180, Error: 2.1894546251764124
Epoch 190, Error: 2.182251496872506
Epoch 200, Error: 2.1735932196188923
Epoch 210, Error: 2.1652786408842113
Epoch 220, Error: 2.1569509791065067
Epoch 230, Error: 2.1479042233422474
Epoch 240, Error: 2.1408255575088866
Epoch 250, Error: 2.1314696119209025
Epoch 260, Error: 2.1217412437361793
Epoch 270, Error: 2.11

## Búsqueda de parámetros

#### Grid Searh para ajustar complejitud de 1 capa oculta

In [13]:

epochs = 200
layer_sizes = [input_size] + hidden_layers + [output_size]
lr = 0.05
for i in range(0, 5):  # Cambia el 5 por el número de potencias de 10 que desees
    val = pow(10, i)
    hidden_layers = [val]  # Tamaños de las capas ocultas
    print(f"layer size: {val}")
    layer_sizes = [input_size] + hidden_layers + [output_size]
    print(layer_sizes)

    model = MultiLayerNetwork(layer_sizes)
    errors = model.train(X_train, y_train, epochs, lr)

    #Evaluar
    predictions = model.predict(X_train)
    accuracy = cp.mean(predictions.argmax(axis=1) == y_train)
    print(f'Test Accuracy: {accuracy}')
    print("")

layer size: 1
[3072, 1, 10]
Epoch 0, Error: 2.302621414426933
Epoch 10, Error: 2.3025529936149853
Epoch 20, Error: 2.302498165861214
Epoch 30, Error: 2.302448581355747
Epoch 40, Error: 2.3024037301658726
Epoch 50, Error: 2.3023631576199195
Epoch 60, Error: 2.302326454222536
Epoch 70, Error: 2.302293250202761
Epoch 80, Error: 2.3022632113558354
Epoch 90, Error: 2.302236035506209
Epoch 100, Error: 2.302211449387694
Epoch 110, Error: 2.302189205853871
Epoch 120, Error: 2.30216908136942
Epoch 130, Error: 2.3021508737480465
Epoch 140, Error: 2.3021344001100186
Epoch 150, Error: 2.302119495036605
Epoch 160, Error: 2.3021060089017698
Epoch 170, Error: 2.3020938063636947
Epoch 180, Error: 2.302082765000622
Epoch 190, Error: 2.3020727740770814
Test Accuracy: 0.1069423929098966

layer size: 10
[3072, 10, 10]
Epoch 0, Error: 2.579213115582387
Epoch 10, Error: 2.3516751832167127
Epoch 20, Error: 2.333705394466311
Epoch 30, Error: 2.322258709739571
Epoch 40, Error: 2.314787074289977
Epoch 50, Error

KeyboardInterrupt: 

### Grid Search para ajustar numero de capas ocultas

In [18]:

num_classes = 10
input_size = 3072
hidden_layer_size = 256
hidden_layers = []  # Tamaños de las capas ocultas
output_size = 10
layer_sizes = [input_size] + hidden_layers + [output_size]
epochs = 100
lr = 0.0001

for i in range(0, 10):  # Cambia el 5 por el número de potencias de 10 que desees
    hidden_layers.append(hidden_layer_size)  # Tamaños de las capas ocultas
    print(f"hidden layers: {hidden_layers}")
    layer_sizes = [input_size] + hidden_layers + [output_size]

    model = MultiLayerNetwork(layer_sizes)
    errors = model.train(X_train, y_train, epochs, lr)

    #Evaluar
    predictions = model.predict(X_train)
    accuracy = cp.mean(predictions.argmax(axis=1) == y_train)
    print(f'Test Accuracy: {accuracy}')
    print("")

hidden layers: [256]
Epoch 0, Error: 105.86025886841283
Test Accuracy: 0.09698996655518395

hidden layers: [256, 256]
Epoch 0, Error: 108.85456644116445
Test Accuracy: 0.14046822742474915

hidden layers: [256, 256, 256]
Epoch 0, Error: 105.7057006341719
Test Accuracy: 0.14046822742474915

hidden layers: [256, 256, 256, 256]
Epoch 0, Error: 110.30727016766471
Test Accuracy: 0.14046822742474915

hidden layers: [256, 256, 256, 256, 256]
Epoch 0, Error: 125.7849884262953
Test Accuracy: 0.14046822742474915

hidden layers: [256, 256, 256, 256, 256, 256]
Epoch 0, Error: 111.10845764177878
Test Accuracy: 0.14046822742474915

hidden layers: [256, 256, 256, 256, 256, 256, 256]
Epoch 0, Error: 106.9112295635407
Test Accuracy: 0.14046822742474915

hidden layers: [256, 256, 256, 256, 256, 256, 256, 256]
Epoch 0, Error: 107.25215446902249
Test Accuracy: 0.14046822742474915

hidden layers: [256, 256, 256, 256, 256, 256, 256, 256, 256]
Epoch 0, Error: 105.26643352242563
Test Accuracy: 0.14046822742474

### Modelo entrenado con parámetros encontrados

## Regularizaciones:

#### Dropout


In [14]:

class MultiLayerNetwork_Dropout:
    def __init__(self, layer_sizes, dropout_rate=0.5):
        self.layer_sizes = layer_sizes
        self.dropout_rate = dropout_rate
        self.weights = []
        self.biases = []
        self.dropout_masks = []

        # Initialize weights and biases for each layer
        for i in range(len(layer_sizes) - 1):
            weight = cp.random.randn(layer_sizes[i], layer_sizes[i + 1]) * cp.sqrt(2. / layer_sizes[i])
            bias = cp.zeros((1, layer_sizes[i + 1]))
            self.weights.append(weight)
            self.biases.append(bias)
            # Initialize dropout mask
            self.dropout_masks.append(None)

    def forward(self, inputs, training=True):
        self.activations = [inputs]
        a = inputs

        for i in range(len(self.weights) - 1):
            z = cp.dot(a, self.weights[i]) + self.biases[i]
            a = sigmoid(z)

            if training:
                # Apply dropout
                dropout_mask = (cp.random.rand(*a.shape) > self.dropout_rate) / (1.0 - self.dropout_rate)
                a *= dropout_mask
                self.dropout_masks[i] = dropout_mask

            self.activations.append(a)

        # Output uses softmax for multiclass classification
        z = cp.dot(a, self.weights[-1]) + self.biases[-1]
        a = softmax(z)
        self.activations.append(a)
        return a

    def predict(self, inputs):
        inputs = cp.array(inputs, ndmin=2)
        return self.forward(inputs, training=False)

    def backward(self, targets, learning_rate):
        m = targets.shape[0]  # number of training examples
        delta_weights = [0] * len(self.weights)
        delta_biases = [0] * len(self.biases)

        # Calculate the initial error (difference between prediction and target for the output layer)
        error = self.activations[-1] - targets

        for i in reversed(range(len(self.weights))):
            # Calculate the delta for the current layer
            delta = error
            delta_weights[i] = cp.dot(self.activations[i].T, delta) / m
            delta_biases[i] = cp.sum(delta, axis=0, keepdims=True) / m

            if i != 0:
                # Propagate the error to the previous layer
                error = cp.dot(delta, self.weights[i].T) * sigmoid_derivative(self.activations[i])
                if self.dropout_masks[i-1] is not None:
                    error *= self.dropout_masks[i-1]

            # Update weights and biases
            self.weights[i] -= learning_rate * delta_weights[i]
            self.biases[i] -= learning_rate * delta_biases[i]

    def train(self, inputs, targets, epochs, learning_rate):
        targets = one_hot(targets, 10)

        errors = []

        for epoch in range(epochs):
            predictions = self.forward(inputs, training=True)
            error = cross_entropy_loss(predictions, targets)
            self.backward(targets, learning_rate)
            errors.append(error)

            if epoch % 10 == 0:
                print(f'Epoch {epoch}, Error: {error}')

        return errors

In [18]:
num_classes = 10
input_size = 3072
hidden_layers = [256,256,256]  # Tamaños de las capas ocultas
output_size = 10
layer_sizes = [input_size] + hidden_layers + [output_size]
epochs = 300
learning_rate = 0.05
dropout_rate = 0.5


model = MultiLayerNetwork_Dropout(layer_sizes, dropout_rate)
errors = model.train(X_train, y_train, epochs, learning_rate)
#Evaluar
predictions = model.predict(X_train)
accuracy = cp.mean(predictions.argmax(axis=1) == y_train)
print(f'Test Accuracy: {accuracy}')
print("")

Epoch 0, Error: 2.8347622914310113
Epoch 10, Error: 2.584034653545719
Epoch 20, Error: 2.576708561948397
Epoch 30, Error: 2.5634259828688353
Epoch 40, Error: 2.5444277988779636
Epoch 50, Error: 2.5514630365111057
Epoch 60, Error: 2.5442052437184186
Epoch 70, Error: 2.5246153301062706
Epoch 80, Error: 2.5304585828005255
Epoch 90, Error: 2.5142585998873153
Epoch 100, Error: 2.5113063027825686
Epoch 110, Error: 2.5180615571681253
Epoch 120, Error: 2.500633209364342
Epoch 130, Error: 2.487416893284751
Epoch 140, Error: 2.4892244982677316
Epoch 150, Error: 2.4845740738069035
Epoch 160, Error: 2.4687288180765274
Epoch 170, Error: 2.4663125528498084
Epoch 180, Error: 2.4636310912035695
Epoch 190, Error: 2.4667570054959995
Epoch 200, Error: 2.449703402118532
Epoch 210, Error: 2.4465359270670812
Epoch 220, Error: 2.4483331509194146
Epoch 230, Error: 2.428616256642728
Epoch 240, Error: 2.430787115031145
Epoch 250, Error: 2.4208043288391305
Epoch 260, Error: 2.4179419673853
Epoch 270, Error: 2.43

### Feature engineering

#### Porcentaje para cada canal RGB

In [43]:
red_pixels = []
green_pixels = []
blue_pixels = []
train_data = train_data[:, 1:-1]

# Iterate through each row in the original matrix
for row in train_data:
    pr = []
    pg = []
    pb = []
    for i in range(len(row)):
        if i % 3 == 0:
            pr.append(row[i])
        
        if i % 3 == 1:
            pg.append(row[i])

        if i % 3 == 2:
            pb.append(row[i])

    red_pixels.append(pr)
    green_pixels.append(pg)
    blue_pixels.append(pb)


#### Más datos

In [44]:
#intensidad media por conal
mean_red = np.mean(red_pixels, axis=1)
mean_green = np.mean(green_pixels, axis=1)
mean_blue = np.mean(blue_pixels, axis=1)

#varianza
var_red = np.var(red_pixels, axis=1)
var_green = np.var(green_pixels, axis=1)
var_blue = np.var(blue_pixels, axis=1)

#desviación estándar
std_red = np.std(red_pixels, axis=1)
std_green = np.std(green_pixels, axis=1)
std_blue = np.std(blue_pixels, axis=1)

#contraste
contrast_red = np.ptp(red_pixels, axis=1) # ptp: peak to peak (max - min)
contrast_green = np.ptp(green_pixels, axis=1)
contrast_blue = np.ptp(blue_pixels, axis=1)

channel_data = np.column_stack((
                                mean_red, mean_green, mean_blue,
                                var_red, var_green, var_blue,
                                std_red, std_green, std_blue,
                                contrast_red, contrast_green, contrast_blue
                                ))

print(channel_data.shape)



(10155, 12)


### Train modelo con dataset de parámetros por imagen

In [47]:
#Cortar en features y labales
channel_data_train_samples = channel_data.shape[0]
features = channel_data  # Features for training

features = cp.array(features)

print(channel_data.shape)
print(features.shape )
print(labels.shape)

X_train = cp.array(features)

num_classes = 10
input_size = 12
hidden_layers = [256,256]  # Tamaños de las capas ocultas
output_size = 10
layer_sizes = [input_size] + hidden_layers + [output_size]
epochs = 1000
learning_rate = 0.1
dropout_rate = 0.2

model = MultiLayerNetwork(layer_sizes)
errors = model.train(X_train, y_train, epochs, learning_rate)
#Evaluar
predictions = model.predict(X_train)

accuracy = cp.mean(predictions.argmax(axis=1) == y_train)
print(f'Test Accuracy: {accuracy}')

(10155, 12)
(10155, 12)
(10155, 1)
Epoch 0, Error: 2.3898330489527955
Epoch 1, Error: 2.305420540347304
Epoch 2, Error: 2.2993254481969383
Epoch 3, Error: 2.2980342028294842
Epoch 4, Error: 2.296853625997203
Epoch 5, Error: 2.295580123332472
Epoch 6, Error: 2.294407309628787
Epoch 7, Error: 2.2933180946727147
Epoch 8, Error: 2.2921351846705575
Epoch 9, Error: 2.2909355283446677
Epoch 10, Error: 2.2898134129250827
Epoch 11, Error: 2.2887518566303573
Epoch 12, Error: 2.287658902915746
Epoch 13, Error: 2.2867520190151156
Epoch 14, Error: 2.285809374551954
Epoch 15, Error: 2.284745352687388
Epoch 16, Error: 2.2839174179325687
Epoch 17, Error: 2.2829906714130854
Epoch 18, Error: 2.282192554418014
Epoch 19, Error: 2.2814327463800157
Epoch 20, Error: 2.28057490864808
Epoch 21, Error: 2.279695240690646
Epoch 22, Error: 2.2790111759546567
Epoch 23, Error: 2.2780190170273493
Epoch 24, Error: 2.277354033945844
Epoch 25, Error: 2.27640258123909
Epoch 26, Error: 2.275621141128945
Epoch 27, Error: 2

## UTIL


### Data prep pero cortando 20% para test

In [ ]:
#Obtener datos (estoy usando pd porque anda considerablemente más rápido que np)
train_data = pd.read_csv(r'C:/Users/kueru/Documents/VSCode/semestre_9/Deep_Learning/T2/train_data_2.csv')
train_data = train_data.to_numpy()

#Cortar en features y labales
train_samples = train_data.shape[0]
features = train_data[:train_samples, :-1]  # Features for training
labels = train_data[:train_samples, -1]   # Labels for training

features = cp.array(features)
labels = cp.array(labels, ndmin=2)
labels = labels.reshape(-1, 1)  # Reshape to (299, 1)

print(train_data.shape)
print(features.shape )
print(labels.shape)


indices = cp.arange(train_samples)

#cortar 20% test 
test_size = 0.2
test_samples = int(test_size * train_samples)
train_indices, test_indices = indices[test_samples:], indices[:test_samples]

X_train, X_test = features[train_indices], features[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]

train_indices = cp.array(train_indices)
test_indices = cp.array(test_indices)
X_train = cp.array(X_train)
X_test = cp.array(X_test)
y_train = cp.array(y_train)
y_test = cp.array(y_test)

In [28]:
#Obtener datos (estoy usando pd porque anda considerablemente más rápido que np)
train_data = pd.read_csv(r'C:/Users/kueru/Documents/VSCode/semestre_9/Deep_Learning/T2/train_data.csv')
train_data = train_data.to_numpy()
    
#Cortar en features y labales
train_samples = train_data.shape[0]
features = train_data[:train_samples, 1:-1]  # Features for training
labels = train_data[:train_samples, -1]   # Labels for training

# Reduce pixel values
features = features / 255.0 
# flatten the label values
labels = labels.flatten()

features = cp.array(features)
labels = cp.array(labels, ndmin=2)
labels = labels.reshape(-1, 1)  # Reshape to (299, 1)

print(train_data.shape)
print(features.shape )
print(labels.shape)


indices = cp.arange(train_samples)


X_train = cp.array(features)

num_classes = 10
input_size = 3072
hidden_layers = [100, 100]  # Tamaños de las capas ocultas
output_size = 10
layer_sizes = [input_size] + hidden_layers + [output_size]

model = MultiLayerNetwork(layer_sizes)
epochs = 1
learning_rate = 0.001

errors = model.train(X_train, y_train, epochs, learning_rate, 2.5)
print(errors)
#Evaluar
predictions = model.predict(X_train)
accuracy = cp.mean(predictions.argmax(axis=1) == y_train)
print(f'Test Accuracy: {accuracy}')

KeyboardInterrupt: 